In [77]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

In [78]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [79]:
print(Base.metadata)
Base.registry

MetaData()


### Mapped classes

In [80]:
from sqlalchemy.orm import Mapped, mapped_column
from sqlalchemy import String, ForeignKey
from typing import Optional


class Cliente(Base):
    __tablename__ = 'CLIENTE'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    sobrenome: Mapped[Optional[str]]

    def __repr__(self) -> str:
        return f'Cliente(id={self.id!r}, nome={self.nome!r}, sobrenome={self.sobrenome!r})'

In [81]:
class Conta(Base):
    __tablename__ = 'CONTA'

    id: Mapped[int] = mapped_column(primary_key=True)
    saldo: Mapped[float] = mapped_column(default=0)
    id_cliente = mapped_column(ForeignKey('CLIENTE.id'))

    def __repr__(self) -> str:
        return f'Conta(id={self.id!r}, saldo={self.saldo!r}, id_cliente={self.id_cliente!r})'

In [82]:
Base.metadata.create_all(engine)

2023-11-26 18:00:57,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:00:57,660 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CLIENTE")
2023-11-26 18:00:57,664 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:00:57,670 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CLIENTE")
2023-11-26 18:00:57,677 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:00:57,680 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CONTA")
2023-11-26 18:00:57,685 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:00:57,689 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CONTA")
2023-11-26 18:00:57,692 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:00:57,696 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CLIENTE" (
	id INTEGER NOT NULL, 
	nome VARCHAR(50) NOT NULL, 
	sobrenome VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-26 18:00:57,698 INFO sqlalchemy.engine.Engine [no key 0.00228s] ()
2023-11-26 18:00:57,702 INFO sqlalchemy.engine.Engine 
CREATE T

In [83]:
from sqlalchemy import inspect

inspect(engine).get_table_names()

2023-11-26 18:00:57,741 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:00:57,745 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-11-26 18:00:57,749 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-26 18:00:57,754 INFO sqlalchemy.engine.Engine ROLLBACK


['CLIENTE', 'CONTA']

In [84]:
alberto = Cliente(nome='alberto', sobrenome='frigatto')
alberto

Cliente(id=None, nome='alberto', sobrenome='frigatto')

In [85]:
gustavo = Cliente(nome='gustavo')
gustavo

Cliente(id=None, nome='gustavo', sobrenome=None)

In [86]:
from sqlalchemy.orm import Session

session = Session(engine, expire_on_commit=False)
session

In [87]:
session.add(alberto)
session.add(gustavo)

In [88]:
session.new

IdentitySet([Cliente(id=None, nome='alberto', sobrenome='frigatto'), Cliente(id=None, nome='gustavo', sobrenome=None)])

In [89]:
session.flush()

2023-11-26 18:00:57,977 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:00:57,984 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-26 18:00:57,987 INFO sqlalchemy.engine.Engine [generated in 0.00051s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('alberto', 'frigatto')
2023-11-26 18:00:57,991 INFO sqlalchemy.engine.Engine INSERT INTO "CLIENTE" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-26 18:00:57,995 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('gustavo', None)


In [90]:
gustavo

Cliente(id=2, nome='gustavo', sobrenome=None)

In [91]:
session.get(Cliente, 2)

Cliente(id=2, nome='gustavo', sobrenome=None)

In [92]:
session.commit()
session.close()

2023-11-26 18:00:58,179 INFO sqlalchemy.engine.Engine COMMIT


In [93]:
alberto.id

1

In [94]:
gustavo.id

2

In [95]:
from sqlalchemy import select

print(select(Cliente))

SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"


In [96]:
with Session(engine) as session:
    result = session.scalars(select(Cliente))

    for cliente in result:
        print(f'{cliente.id} -> {cliente.nome} -> {cliente.sobrenome}')

2023-11-26 18:00:58,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-26 18:00:58,419 INFO sqlalchemy.engine.Engine SELECT "CLIENTE".id, "CLIENTE".nome, "CLIENTE".sobrenome 
FROM "CLIENTE"
2023-11-26 18:00:58,421 INFO sqlalchemy.engine.Engine [generated in 0.00206s] ()
1 -> alberto -> frigatto
2 -> gustavo -> None
2023-11-26 18:00:58,426 INFO sqlalchemy.engine.Engine ROLLBACK
